In [9]:
import numpy as np
from PIL import Image
import joblib  # veya pickle
import os
import cv2
import numpy as np
import pickle
from skimage.feature import hog
from google.colab import drive
import zipfile


In [10]:
# Google Drive'ı bağla
drive.mount('/content/drive')

# Dataset 1: Fresh_and_Rotten_Classification
zip_path1 = "/content/drive/MyDrive/Colab Notebooks/Fresh_and_Rotten_Classification.zip"
extract_path1 = "/content/Fresh_and_Rotten_Classification"

if not os.path.exists(extract_path1):
    with zipfile.ZipFile(zip_path1, 'r') as zip_ref:
        zip_ref.extractall(extract_path1)
    print(f"dataset1 çıkartıldı: {extract_path1}")
else:
    print(f"dataset1 zaten mevcut, tekrar çıkartılmadı: {extract_path1}")

# Dataset 2: Fruit_And_Vegetable_Diseases_Dataset
zip_path2 = "/content/drive/MyDrive/Colab Notebooks/Fruit_And_Vegetable_Diseases_Dataset.zip"
extract_path2 = "/content/Fruit_And_Vegetable_Diseases_Dataset"

if not os.path.exists(extract_path2):
    with zipfile.ZipFile(zip_path2, 'r') as zip_ref:
        zip_ref.extractall(extract_path2)
    print(f"dataset2 çıkartıldı: {extract_path2}")
else:
    print(f"dataset2 zaten mevcut, tekrar çıkartılmadı: {extract_path2}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
dataset1 zaten mevcut, tekrar çıkartılmadı: /content/Fresh_and_Rotten_Classification
dataset2 çıkartıldı: /content/Fruit_And_Vegetable_Diseases_Dataset


In [12]:
def extract_hsv(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv_hist = cv2.calcHist(
        [hsv], [0, 1, 2], None,
        [16, 16, 16],
        [0, 180, 0, 256, 0, 256]
    ).flatten()
    return hsv_hist

def extract_hog(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    hog_feat = hog(
        gray,
        orientations=6,
        pixels_per_cell=(32, 32),
        cells_per_block=(2, 2),
        block_norm='L2-Hys',
        feature_vector=True
    )
    return hog_feat

def build_single_feature(image_path, fitted_pca):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Görüntü okunamadı: {image_path}")
    img = cv2.resize(img, (128, 128))

    hsv_feat = extract_hsv(img)
    hog_feat = extract_hog(img)
    hog_pca  = fitted_pca.transform(hog_feat.reshape(1, -1))
    x_final  = np.concatenate([hog_pca, hsv_feat.reshape(1, -1)], axis=1)
    return x_final

def predict_single(image_path, model_pkl_path, pca_pkl_path, topk=5):

    with open(model_pkl_path, "rb") as f:
        model = pickle.load(f)
    with open(pca_pkl_path, "rb") as f:
        pca = pickle.load(f)

    X = build_single_feature(image_path, pca)

    y_pred = model.predict(X)[0]
    if hasattr(model, "predict_proba"):
        proba = model.predict_proba(X)[0]
        classes = model.classes_

        idxs = np.argsort(proba)[::-1][:topk]
        top = [(classes[i], float(proba[i])) for i in idxs]
    else:
        top = [(y_pred, None)]

    return y_pred, top


In [11]:
model_pkl_path = "/content/drive/MyDrive/Colab Notebooks/saved_models/rf_fresh_and_rotten.pkl"
pca_pkl_path = "/content/drive/MyDrive/Colab Notebooks/saved_models/pca_fresh_and_rotten.pkl"
image_path = "/content/drive/MyDrive/Colab Notebooks/Demo/Fresh_and_Rotten_Classification/freshbanana/b_f057.png"

In [13]:
pred, top_probs = predict_single(image_path, model_pkl_path, pca_pkl_path, topk=5)
print("Tahmin:", pred)
print("Top-5:", top_probs)

Tahmin: freshbanana
Top-5: [(np.str_('freshbanana'), 0.9299551591332943), (np.str_('rottenoranges'), 0.03), (np.str_('rottenapples'), 0.020012996579444414), (np.str_('freshoranges'), 0.01), (np.str_('freshpotato'), 0.01)]


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished


In [14]:
model_pkl_path = "/content/drive/MyDrive/Colab Notebooks/saved_models/rf_fruit_and_vegetable.pkl"
pca_pkl_path = "/content/drive/MyDrive/Colab Notebooks/saved_models/pca_fruit_and_vegetable.pkl"

In [15]:
pred, top_probs = predict_single(image_path, model_pkl_path, pca_pkl_path, topk=5)
print("Tahmin:", pred)
print("Top-5:", top_probs)

Tahmin: Banana__Healthy
Top-5: [(np.str_('Banana__Healthy'), 0.9539563378677611), (np.str_('Apple__Rotten'), 0.03173936730169738), (np.str_('Orange__Healthy'), 0.012195568082242349), (np.str_('Apple__Healthy'), 0.0007444204274499519), (np.str_('Banana__Rotten'), 0.0005535327744974456)]


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished
